# Basic scene generation and visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

import sys
sys.path.append('..')
from dsynth import PACKAGE_DIR
import os
os.chdir(PACKAGE_DIR.parent) # change working dir

## Assets

In [ ]:
from hydra import compose, initialize

from dsynth.scene_gen.hydra_configs import DsContinuousConfig, ShelfConfig, FillingType
from dsynth.scene_gen.scene_generator import SceneGeneratorContinuous

from dsynth.assets.asset import load_assets_lib
from dsynth.scene_gen.utils import flatten_dict

Asset configs are stored in `conf/assets` directory using Hydra configs. You can load them using `load_assets_lib`:

In [ ]:
# load assets/assets_preprocessed.yaml config
with initialize(version_base=None, config_path='../conf' ):
    assets_cfg = compose(config_name="assets/assets_preprocessed")
    
assets_lib = load_assets_lib(assets_cfg, disable_caching=True)

All product items are stored in `assets_lib['assets']['products_hierarchy']['food']`. They are grouped into several types.

In [ ]:
assets_lib['assets']['products_hierarchy']['food'].keys()

Each item is represented as an instance of the abstract class `Asset`. It stores asset path, scaling parameters and other meta information:

In [ ]:
assets_lib['assets']['products_hierarchy']['food']['CANNED']['ArgonautAlaskaSalmon']

Another important part of asset library is `fixtures` which stores scene assets, such as empty shelvings (`shelf`, `shelf_metal`, etc.), filled shelvings (a.k.a. "fake" - `shelf_fake_1`, `shelf_metal_fake_1`, etc.) and other non-active scene objects (`pile_of_pallets`, `pallets_and_boxes`).

In [ ]:
assets_lib['assets']['fixtures'].keys()

## Scene generation

Scene synthesis is described using Hydra configs, such as (`conf/config_continuous.yaml`). It consists of two parts:
* assets config (such as `assets/assets_preprocessed.yaml` - mentioned above)
* ds-configs (such as `conf/ds_continuous/config.yaml`) which describes layout, and arrangement information

Let's look at base ds-config `conf/ds_continuous/config.yaml`:

In [ ]:
from hydra.core.config_store import ConfigStore

In [ ]:
cs = ConfigStore.instance()
cs.store(group="shelves", name="base_shelf_config", node=ShelfConfig)
cs.store(group="ds_continuous", name="main_darkstore_continuous_config_base", node=DsContinuousConfig)
with initialize(version_base=None, config_path='../conf' ):
    base_scene_config = compose(config_name="ds_continuous/config")

In [ ]:
base_scene_config

As we can see, it contains the name of the scene, its size, layout and arrangement generation parameters. See config class `dsynth.scene_gen.hydra_configs.DsContinuousConfig` for more details.

Let's make our custom config `custom_scene.yaml` in `conf/ds_continuous` directory. We inherit from base config, change scene sizes and object arrangements (see below):

In [ ]:
SCENE_DIR = "generated_envs/custom_scene"

In [ ]:
custom_scene_config = f"""
defaults:
  - main_darkstore_continuous_config_base   # inherit from base config
  - _self_

name: 'custom_scene'
output_dir: {SCENE_DIR}

# scene size (in meters)
size_x: 8
size_y: 6

# Number of scenes to generate
num_scenes: 4

# Number of workers (the more workers, the faster generation and more RAM-expensive)
num_workers: 2

# use tensor field -based algorithm for layout generation
layout_gen_type: PROCEDURAL_TENSOR_FIELD

# layout / arrangement randomization
randomize_layout: true
randomize_arrangements: true


random_seed: 42

# list of active shelving
active_shelvings_list:

  # Active shelf config
  - name: 'mixed_shelf'
      
    # place products in columns as specified in `board_product_numcol` parameter
    filling_type: BOARDWISE_COLUMNS 

    # adjust gaps between items
    x_gap: 0.06
    y_gap: 0.01

    # shelf asset
    shelf_asset: fixtures.shelf_metal
    
    # whether to shuffle board arrangements before new scene generation
    shuffle_boards: False                       
    
    board_product_numcol:
      1:                                           # arrangement fot the 1st board
        food.dairy_products.milkCarton: 2            # place milk in 2 columns
      2:                                           # arrangement fot the 2nd board
        food.DRINKS_SODA.FantaSaborNaranja2L: 6   # place fanta in 6 columns
      3:                                           # arrangement fot the 3rd board
        food.drinks.coffeePackaging: 1

# non-active shelving assets to be placed
inactive_shelvings_list:
  - fixtures.shelf_metal
  - fixtures.small_shelf_two_sided

# non-active wall shelving assets to be placed
inactive_wall_shelvings_list:
  - fixtures.freezer_large_filled

# non-active surrounding assets to be placed
scene_fixtures_list: 
  - fixtures.pile_of_pallets

# Fake shelves are inactive assets of cluttered shelves. 
# Needed to create visible fullness 

fake_arrangements_mapping:

  # Change every `fixtures.shelf_metal` randomly to one of 
  # `fixtures.shelf_metal_fake_1` or `fixtures.shelf_metal_fake_2`
  # to create visible fullness
  
  fixtures.shelf_metal:
    - fixtures.shelf_metal_fake_1
    - fixtures.shelf_metal_fake_2

  # Change every `fixtures.small_shelf_two_sided` randomly to one of 
  # `fixtures.small_shelf_two_sided_fake_1` or `fixtures.small_shelf_two_sided_fake_2`
  
  fixtures.small_shelf_two_sided:
    - fixtures.small_shelf_two_sided_fake_1
    - fixtures.small_shelf_two_sided_fake_2
"""

In [ ]:
with open("conf/ds_continuous/custom_scene.yaml", "w") as f:
    f.write(custom_scene_config)

Check `conf/ds_continuous/custom_scene.yaml` and read comments.

In [ ]:
with initialize(version_base=None, config_path='../conf' ):
    scene_config = compose(config_name="config_continuous", overrides=["ds_continuous=custom_scene", "assets=assets_preprocessed"])

## Layout generation

Now it is easy to generate scenes using our custom config.

In [ ]:
from dsynth.scene_gen.scene_generator import SceneGeneratorContinuous

In [ ]:
generator = SceneGeneratorContinuous(scene_config, scene_config.ds_continuous.output_dir)

In [ ]:
results = generator.generate()

In [ ]:
results = np.array(results)
if np.all(results):
    print(f"Done")
elif np.all(~results):
    print(f"All generations are failed")
else:
    print(f"Not all generations are successful: {results}")

Let's visualize layouts of generated scenes.

In [ ]:
layout_img_paths = list(Path(scene_config.ds_continuous.output_dir).glob('*.jpg'))
for i in range(4):
    plt.subplot(2, 2, i + 1)
    layout_img = plt.imread(layout_img_paths[i])
    plt.imshow(layout_img)

The entire generation process above can be done using `scripts/generate_scene_continuous.py`:

```bash
python scripts/generate_scene_continuous.py ds_continuous=custom_scene assets=assets_preprocessed
```

See bash script `bash/generate_scenes.sh` for generating all the training scenes.

## Load the scene inside ManiSkill environment

Let's load our scenes inside ManiSkill. Let's use `PickToBasketContEnv` environment as an example.

In [ ]:
import gymnasium as gym
import mani_skill.envs
from dsynth.envs import *
from dsynth.robots import *

In [ ]:
print(PickToBasketContEnv.__doc__)

In [ ]:
env = gym.make('PickToBasketContEnv', 
       config_dir_path = SCENE_DIR,
       robot_uids='ds_fetch_basket', 
       render_mode='rgb_array', 
       enable_shadow=True,
       obs_mode='rgbd',
       )

We can check all present items in `scene_items.csv`:

In [ ]:
products_df = pd.read_csv(Path(SCENE_DIR) / 'scene_items.csv')
products_df

**Note:** flag `reconfigure=True` should always be specified during resetting

In [ ]:
obs, info = env.reset(seed=42, options={'reconfigure': True})

Language instruction:

In [ ]:
bytes(obs['extra']['language_instruction_bytes'][0]).decode()

Sensor data:

In [ ]:
plt.subplot(131)
plt.imshow(obs['sensor_data']['left_base_camera_link']['rgb'][0].cpu().numpy())
plt.subplot(132)
plt.imshow(obs['sensor_data']['right_base_camera_link']['rgb'][0].cpu().numpy())
plt.subplot(133)
plt.imshow(obs['sensor_data']['fetch_hand']['rgb'][0].cpu().numpy())

Human render:

In [ ]:
render = env.render()
plt.imshow(render[0].cpu().numpy())

## Motion Planning

We have Fanta bottles in the shelf. Let's use `PickToBasketContFantaEnv` environment and motion planning solver to collect demonstration trajectory.

In [ ]:
ENV_ID = 'PickToBasketContFantaEnv'

In [ ]:
from mani_skill.utils.wrappers.record import RecordEpisode

from dsynth.planning import MP_SOLUTIONS
from dsynth.planning.utils import BAD_ENV_ERROR_CODE

env = gym.make(ENV_ID, 
       config_dir_path = SCENE_DIR,
        control_mode="pd_joint_pos",
       robot_uids='ds_fetch_basket', 
       render_mode="rgb_array", 
       enable_shadow=True,
       obs_mode='none',
       )

env = RecordEpisode(
        env,
        output_dir=os.path.join(SCENE_DIR, "motionplanning"),
        save_video=True,
        source_type="motionplanning",
        source_desc="official motion planning solution from dsynth contributors",
        video_fps=30,
        record_reward=False,
        save_on_reset=False
    )

In [ ]:
mp_solver = MP_SOLUTIONS[ENV_ID]

In [ ]:
seed = 0
success = False
MAX_TRIES = 10

for n_try in range(MAX_TRIES):
    print(f"Try {n_try}")
    
    try:
        res = mp_solver(env, seed=seed, debug=False, vis=False)
    except Exception as e:
        print(f"Cannot find valid solution because of an error in motion planning solution: {e}")
        res = -1
        
    if res == BAD_ENV_ERROR_CODE:
        print(f"Bad environment! Skipping...")
        seed += 1

    elif res != -1:
        success = True
        env.flush_trajectory()
        env.flush_video()
        break

if not success:
    print("Solver can't do the task")
else:
    print("Success")

In [ ]:
from IPython.display import Video
Video(f"{SCENE_DIR}/motionplanning/0.mp4", embed=True)
